<a href="https://colab.research.google.com/github/zafe312/public_zafe/blob/master/ML_eduonix/Diabetes%20prediction%20using%20grid%20search/Diabetes_onset_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import pandas
import numpy
import sklearn
import keras

print('Python: {}'.format(sys.version))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))
print('Sklearn: {}'.format(sklearn.__version__))
print('Keras: {}'.format(keras.__version__))
print(keras.backend.backend()) # make sure this is theano and not tensorflow

Using TensorFlow backend.


Python: 3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]
Pandas: 1.0.5
Numpy: 1.18.5
Sklearn: 0.22.2.post1
Keras: 2.3.1
tensorflow


In [ ]:
# import os
# os.environ["KERAS_BACKEND"] = "theano"
# import keras.backend
# keras.backend.set_image_dim_ordering('th')

In [ ]:
# print(open('.keras/keras.json').read())

In [ ]:
#!pip show keras

In [3]:
import pandas as pd
import numpy as np

# import the uci pima indians diabetes dataset
# url = "http://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data" # no longer available
names = ['n_pregnant', 'glucose_concentration', 'blood_pressuer (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)',
        'BMI', 'pedigree_function', 'age', 'class']
df = pd.read_csv('diabetes.data.txt', names = names)

In [4]:
df[:5]

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.describe()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [6]:
df[df['glucose_concentration']==0]

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
75,1,0,48,20,0,24.7,0.140,22,0
182,1,0,74,20,23,27.7,0.299,21,0
342,1,0,68,35,0,32.0,0.389,22,0
349,5,0,80,32,0,41.0,0.346,37,1
502,6,0,68,41,0,39.0,0.727,41,1


In [7]:
# preprocessing the data, mark zero values as NaN and drop
columns = ['glucose_concentration', 'blood_pressuer (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)', 'BMI']

for col in columns:
    df[col].replace(0, np.NaN, inplace=True)
    
df.describe()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [8]:
df[df['glucose_concentration']==0]

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class


In [9]:
# Drop rows with missing values
df.dropna(inplace=True)

# summarize the number of rows and columns in df
df.describe()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,3.211424,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,0.000000,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [10]:
# Covert df to np array
dataset = df.values

print(dataset)
print(dataset.shape)

[[  1.     89.     66.    ...   0.167  21.      0.   ]
 [  0.    137.     40.    ...   2.288  33.      1.   ]
 [  3.     78.     50.    ...   0.248  26.      1.   ]
 ...
 [  2.     88.     58.    ...   0.766  22.      0.   ]
 [ 10.    101.     76.    ...   0.171  63.      0.   ]
 [  5.    121.     72.    ...   0.245  30.      0.   ]]
(392, 9)


In [11]:
# Split into input X and output Y
X = dataset[:,:8]
Y = dataset[:,8].astype(int)


In [12]:
# Normalize the data using sklearn StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)

In [13]:
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [14]:
# Transform and display the training data
X_standardized = scaler.transform(X)

data = pd.DataFrame(X_standardized)
data.describe()

,0,1,2,3,4,5,6,7
count,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02
mean,-4.021726e-17,3.129583e-17,-4.641624e-16,1.042250e-16,6.485742e-17,1.543550e-16,3.880116e-17,1.028089e-16
std,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00
min,-1.029213e+00,-2.161731e+00,-3.739001e+00,-2.108484e+00,-1.196867e+00,-2.120941e+00,-1.269525e+00,-9.682991e-01
25%,-7.174265e-01,-7.665958e-01,-6.941640e-01,-7.755315e-01,-6.681786e-01,-6.676780e-01,-7.340909e-01,-7.719850e-01
50%,-4.056403e-01,-1.176959e-01,-5.314565e-02,-1.384444e-02,-2.574448e-01,1.621036e-02,-2.131475e-01,-3.793569e-01
75%,5.297185e-01,6.609841e-01,5.878727e-01,7.478426e-01,2.859877e-01,5.718696e-01,4.751644e-01,5.040564e-01
max,4.271153e+00,2.445459e+00,3.151946e+00,3.223325e+00,5.812990e+00,4.846172e+00,5.497667e+00,4.921123e+00


In [15]:
# import necessary sklearn and keras packages
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

In [16]:
# Start defining the model
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

model = create_model()
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
# Do a grid search for the optimal batch size and number of epochs
# import necessary packages
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, verbose = 1)

# define the grid search parameters
batch_size = [10, 20, 40]
epochs = [10, 50, 100]

# make a dictionary of the grid search parameters
param_grid = dict(batch_size=batch_size, epochs=epochs)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed, shuffle=True), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] batch_size=10, epochs=10 ........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/10
313/313 [==============================] - 0s 419us/step - loss: 0.6730 - accuracy: 0.6709
Epoch 2/10
313/313 [==============================] - 0s 103us/step - loss: 0.6468 - accuracy: 0.6805
Epoch 3/10
313/313 [==============================] - 0s 100us/step - loss: 0.6346 - accuracy: 0.6805
Epoch 4/10
313/313 [==============================] - 0s 127us/step - loss: 0.6302 - accuracy: 0.6805
Epoch 5/10
313/313 [==============================] - 0s 102us/step - loss: 0.6281 - accuracy: 0.6805
Epoch 6/10
313/313 [==============================] - 0s 102us/step - loss: 0.6272 - accuracy: 0.6805
Epoch 7/10
313/313 [==============================] - 0s 105us/step - loss: 0.6268 - accuracy: 0.6805
Epoch 8/10
313/313 [==============================] - 0s 105us/step - loss: 0.6275 - accuracy: 0.6805
Epoch 9/10
313/313 [==============================] - 0s 105us/step - loss: 0.6270 - accuracy: 0.6805
Epoch 10/10
79/79 [==============================] - 0s 276us/step
[CV] ..........

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


Epoch 1/10
313/313 [==============================] - 0s 418us/step - loss: 0.6050 - accuracy: 0.6709
Epoch 2/10
313/313 [==============================] - 0s 107us/step - loss: 0.4822 - accuracy: 0.6709
Epoch 3/10
313/313 [==============================] - 0s 109us/step - loss: 0.4667 - accuracy: 0.7444
Epoch 4/10
313/313 [==============================] - 0s 99us/step - loss: 0.4567 - accuracy: 0.8019
Epoch 5/10
313/313 [==============================] - 0s 110us/step - loss: 0.4483 - accuracy: 0.8083
Epoch 6/10
313/313 [==============================] - 0s 121us/step - loss: 0.4363 - accuracy: 0.8147
Epoch 7/10
313/313 [==============================] - 0s 102us/step - loss: 0.4335 - accuracy: 0.8179
Epoch 8/10
313/313 [==============================] - 0s 103us/step - loss: 0.4199 - accuracy: 0.8179
Epoch 9/10
313/313 [==============================] - 0s 106us/step - loss: 0.4147 - accuracy: 0.8211
Epoch 10/10
79/79 [==============================] - 0s 275us/step
[CV] ...........

...

Epoch 1/50
314/314 [==============================] - 0s 437us/step - loss: 0.5900 - accuracy: 0.7643
Epoch 2/50
314/314 [==============================] - 0s 104us/step - loss: 0.4717 - accuracy: 0.7611
Epoch 3/50
314/314 [==============================] - 0s 117us/step - loss: 0.4500 - accuracy: 0.7675
Epoch 4/50
314/314 [==============================] - 0s 117us/step - loss: 0.4470 - accuracy: 0.7739
Epoch 5/50
314/314 [==============================] - 0s 103us/step - loss: 0.4343 - accuracy: 0.7739
Epoch 6/50
314/314 [==============================] - 0s 106us/step - loss: 0.4331 - accuracy: 0.8025
Epoch 7/50
314/314 [==============================] - 0s 100us/step - loss: 0.4207 - accuracy: 0.7866
Epoch 8/50
314/314 [==============================] - 0s 102us/step - loss: 0.4366 - accuracy: 0.7866
Epoch 9/50
314/314 [==============================] - 0s 101us/step - loss: 0.4199 - accuracy: 0.7994
Epoch 10/50
314/314 [==============================] - 0s 106us/step - loss: 0.417

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   14.5s remaining:    0.0s


Epoch 1/50
314/314 [==============================] - 0s 432us/step - loss: 0.5939 - accuracy: 0.6656
Epoch 2/50
314/314 [==============================] - 0s 107us/step - loss: 0.4908 - accuracy: 0.7197
Epoch 3/50
314/314 [==============================] - 0s 106us/step - loss: 0.4706 - accuracy: 0.7866
Epoch 4/50
314/314 [==============================] - 0s 104us/step - loss: 0.4366 - accuracy: 0.7994
Epoch 5/50
314/314 [==============================] - 0s 109us/step - loss: 0.4236 - accuracy: 0.8025
Epoch 6/50
314/314 [==============================] - 0s 103us/step - loss: 0.4114 - accuracy: 0.7898
Epoch 7/50
314/314 [==============================] - 0s 102us/step - loss: 0.4040 - accuracy: 0.8121
Epoch 8/50
314/314 [==============================] - 0s 104us/step - loss: 0.4052 - accuracy: 0.8153
Epoch 9/50
314/314 [==============================] - 0s 107us/step - loss: 0.3970 - accuracy: 0.8153
Epoch 10/50
314/314 [==============================] - 0s 104us/step - loss: 0.394

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  1.4min finished


Epoch 1/50
392/392 [==============================] - 0s 326us/step - loss: 0.6809 - accuracy: 0.6429
Epoch 2/50
392/392 [==============================] - 0s 56us/step - loss: 0.6306 - accuracy: 0.7755
Epoch 3/50
392/392 [==============================] - 0s 58us/step - loss: 0.5875 - accuracy: 0.7806
Epoch 4/50
392/392 [==============================] - 0s 57us/step - loss: 0.5530 - accuracy: 0.7883
Epoch 5/50
392/392 [==============================] - 0s 70us/step - loss: 0.5255 - accuracy: 0.7959
Epoch 6/50
392/392 [==============================] - 0s 61us/step - loss: 0.5043 - accuracy: 0.8061
Epoch 7/50
392/392 [==============================] - 0s 59us/step - loss: 0.4887 - accuracy: 0.8061
Epoch 8/50
392/392 [==============================] - 0s 61us/step - loss: 0.4749 - accuracy: 0.8112
Epoch 9/50
392/392 [==============================] - 0s 60us/step - loss: 0.4629 - accuracy: 0.8061
Epoch 10/50
392/392 [==============================] - 0s 62us/step - loss: 0.4557 - accur

In [22]:
# Do a grid search for learning rate and dropout rate
# import necessary packages
from keras.layers import Dropout

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(learn_rate, dropout_rate):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = learn_rate)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2]

# make a dictionary of the grid search parameters
param_grid = dict(learn_rate=learn_rate, dropout_rate=dropout_rate)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed, shuffle=True), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .. dropout_rate=0.0, learn_rate=0.001, score=0.747, total=   2.1s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s remaining:    0.0s


[CV] .. dropout_rate=0.0, learn_rate=0.001, score=0.671, total=   2.1s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


...

[CV] ... dropout_rate=0.0, learn_rate=0.01, score=0.756, total=   2.3s
[CV] dropout_rate=0.0, learn_rate=0.01 ...............................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   20.2s remaining:    0.0s


[CV] ... dropout_rate=0.0, learn_rate=0.01, score=0.769, total=   2.3s
[CV] dropout_rate=0.0, learn_rate=0.1 ................................
[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.696, total=   2.2s
[CV] dropout_rate=0.0, learn_rate=0.1 ................................
[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.658, total=   2.3s
[CV] dropout_rate=0.0, learn_rate=0.1 ................................
[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.654, total=   2.3s
[CV] dropout_rate=0.0, learn_rate=0.1 ................................
[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.795, total=   2.3s
[CV] dropout_rate=0.0, learn_rate=0.1 ................................
[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.718, total=   2.3s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.1, learn_rate=0.001, score=0.785, total=   2.5s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  1.8min finished


Best: 0.7731580615043641, using {'dropout_rate': 0.2, 'learn_rate': 0.001}
0.7579032897949218 (0.053475722316301606) with: {'dropout_rate': 0.0, 'learn_rate': 0.001}
0.7271989703178405 (0.038758866185635336) with: {'dropout_rate': 0.0, 'learn_rate': 0.01}
0.7042194128036499 (0.051231456126124106) with: {'dropout_rate': 0.0, 'learn_rate': 0.1}
0.7526777029037476 (0.04365905433859848) with: {'dropout_rate': 0.1, 'learn_rate': 0.001}
0.7525478720664978 (0.03809896725786535) with: {'dropout_rate': 0.1, 'learn_rate': 0.01}
0.6965595602989196 (0.04440542869348165) with: {'dropout_rate': 0.1, 'learn_rate': 0.1}
0.7731580615043641 (0.049546689059575204) with: {'dropout_rate': 0.2, 'learn_rate': 0.001}
0.7551769018173218 (0.021080958875180574) with: {'dropout_rate': 0.2, 'learn_rate': 0.01}
0.7577085375785828 (0.035464080079555454) with: {'dropout_rate': 0.2, 'learn_rate': 0.1}


In [25]:
# Do a grid search to optimize kernel initialization and activation functions
# import necessary packages

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(activation, init):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer= init, activation= activation))
    model.add(Dense(4, input_dim = 8, kernel_initializer= init, activation= activation))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
activation = ['softmax', 'relu', 'tanh', 'linear']
init = ['uniform', 'normal', 'zero']

# make a dictionary of the grid search parameters
param_grid = dict(activation = activation, init = init)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed,shuffle=True), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .... activation=softmax, init=uniform, score=0.810, total=   2.3s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.3s remaining:    0.0s


[CV] .... activation=softmax, init=uniform, score=0.684, total=   2.2s
[CV] activation=softmax, init=uniform ................................


...

[CV] ..... activation=softmax, init=normal, score=0.808, total=   2.9s
[CV] activation=softmax, init=normal .................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   20.7s remaining:    0.0s


[CV] ..... activation=softmax, init=normal, score=0.756, total=   2.2s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.620, total=   2.2s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.658, total=   2.2s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.692, total=   2.2s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.692, total=   2.2s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.679, total=   2.3s
[CV] activation=relu, init=uniform ...................................
[CV] ....... activation=relu, init=uniform, score=0.810, total=   2.3s
[CV] activation=relu, init=uniform ...................................
[CV] .

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  2.2min finished


Best: 0.7884128570556641, using {'activation': 'linear', 'init': 'normal'}
0.7679649353027344 (0.04650731524571878) with: {'activation': 'softmax', 'init': 'uniform'}
0.7730606913566589 (0.03652307810269447) with: {'activation': 'softmax', 'init': 'normal'}
0.6685167193412781 (0.027157343185035612) with: {'activation': 'softmax', 'init': 'zero'}
0.767900037765503 (0.0385873127448399) with: {'activation': 'relu', 'init': 'uniform'}
0.7528075337409973 (0.0473190551715507) with: {'activation': 'relu', 'init': 'normal'}
0.6685167193412781 (0.027157343185035612) with: {'activation': 'relu', 'init': 'zero'}
0.7807530045509339 (0.03971707257900321) with: {'activation': 'tanh', 'init': 'uniform'}
0.7781889081001282 (0.0382830820022173) with: {'activation': 'tanh', 'init': 'normal'}
0.6685167193412781 (0.027157343185035612) with: {'activation': 'tanh', 'init': 'zero'}
0.7832846522331238 (0.03328030952820975) with: {'activation': 'linear', 'init': 'uniform'}
0.7884128570556641 (0.034642788843363

In [26]:
# Do a grid search to find the optimal number of neurons in each hidden layer
# import necessary packages

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(neuron1, neuron2):
    # create model
    model = Sequential()
    model.add(Dense(neuron1, input_dim = 8, kernel_initializer= 'uniform', activation= 'linear'))
    model.add(Dense(neuron2, input_dim = neuron1, kernel_initializer= 'uniform', activation= 'linear'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
neuron1 = [4, 8, 16]
neuron2 = [2, 4, 8]

# make a dictionary of the grid search parameters
param_grid = dict(neuron1 = neuron1, neuron2 = neuron2)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed, shuffle=True), refit = True, verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................ neuron1=4, neuron2=2, score=0.785, total=   2.4s
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s remaining:    0.0s


[CV] ................ neuron1=4, neuron2=2, score=0.722, total=   2.2s
[CV] neuron1=4, neuron2=2 ............................................


...

[CV] ................ neuron1=4, neuron2=4, score=0.821, total=   2.1s
[CV] neuron1=4, neuron2=4 ............................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   19.8s remaining:    0.0s


[CV] ................ neuron1=4, neuron2=4, score=0.782, total=   2.1s
[CV] neuron1=4, neuron2=8 ............................................
[CV] ................ neuron1=4, neuron2=8, score=0.797, total=   3.1s
[CV] neuron1=4, neuron2=8 ............................................
[CV] ................ neuron1=4, neuron2=8, score=0.722, total=   2.2s
[CV] neuron1=4, neuron2=8 ............................................
[CV] ................ neuron1=4, neuron2=8, score=0.808, total=   2.2s
[CV] neuron1=4, neuron2=8 ............................................
[CV] ................ neuron1=4, neuron2=8, score=0.821, total=   2.2s
[CV] neuron1=4, neuron2=8 ............................................
[CV] ................ neuron1=4, neuron2=8, score=0.795, total=   2.3s
[CV] neuron1=8, neuron2=2 ............................................
[CV] ................ neuron1=8, neuron2=2, score=0.797, total=   2.3s
[CV] neuron1=8, neuron2=2 ............................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  1.7min finished


Best: 0.7884128570556641, using {'neuron1': 4, 'neuron2': 8}
0.7833171010017395 (0.03406026929476505) with: {'neuron1': 4, 'neuron2': 2}
0.7858487486839294 (0.03454415334118797) with: {'neuron1': 4, 'neuron2': 4}
0.7884128570556641 (0.034642788843363075) with: {'neuron1': 4, 'neuron2': 8}
0.7858487486839294 (0.03454415334118797) with: {'neuron1': 8, 'neuron2': 2}
0.7832846522331238 (0.03328030952820975) with: {'neuron1': 8, 'neuron2': 4}
0.7832846522331238 (0.03328030952820975) with: {'neuron1': 8, 'neuron2': 8}
0.7858487606048584 (0.033579049025109) with: {'neuron1': 16, 'neuron2': 2}
0.7858487486839294 (0.03454415334118797) with: {'neuron1': 16, 'neuron2': 4}
0.7858487486839294 (0.03454415334118797) with: {'neuron1': 16, 'neuron2': 8}


In [27]:
# generate predictions with optimal hyperparameters
y_pred = grid.predict(X_standardized)

In [28]:
print(y_pred.shape)

(392, 1)


In [29]:
print(y_pred[:5])

[[0]
 [1]
 [0]
 [1]
 [1]]


In [30]:
# Generate a classification report
from sklearn.metrics import classification_report, accuracy_score

print(accuracy_score(Y, y_pred))
print(classification_report(Y, y_pred))

0.7831632653061225
              precision    recall  f1-score   support

           0       0.81      0.89      0.85       262
           1       0.72      0.57      0.64       130

    accuracy                           0.78       392
   macro avg       0.76      0.73      0.74       392
weighted avg       0.78      0.78      0.78       392

